# Funkcja process_csv() odpowiedzialna za wczytanie i wstępną obróbkę danych do postaci 5 sekundowych kawałków

In [1]:
from itertools import chain
def process_csv(path, dirty_acceptation=25):
    series = ['ENGRPM', 'FUELUS', 'ENGTPS', 'GROILP', 'SELGEAR', 'SPEED', 'BREAKP', 'TRNAUT', 'HYDOILP']
    f = open(path, 'rt')
    csv_content = f.readlines()
    f.close()
    output = []
    if len(csv_content) > 0 and csv_content[0] == "ASCII\n":
        sep = csv_content[1].replace("\n", "")
        s = 0
        dirty = False
        ts = None
        sample = [[None for i in range(9)] for i in range(5)]
        for line in csv_content:
            line = line.replace("\n", "").split(sep)
            if not dirty:
                for i, j in enumerate(series):
                    if j in line[0]:
                        if ts is None:
                            ts = line[3]
                        if ts != line[3] and s != 4:
                            ts = line[3]
                            s += 1
                        if line[6] != "192":
                            sample[s][i] = None
#                             dirty = True
                        else:
                            sample[s][i] = float(line[5])
                        break
            if 'ENGOILP' in line[0]:
                if line[6] == "192":
                    sample = list(chain(*sample)) + [line[5]]
                else:
                    sample = list(chain(*sample)) + [None]
                if sample.count(None) < dirty_acceptation:
                    output.append(sample)
                s = 0
                dirty = False
                sample = [[None for i in range(9)] for i in range(5)]
        return output
    return output

# wczytywanie przykładowych danych dla LK3 045L

In [2]:
import os
dataset_path = './datasets/LK3 045L'
learning_dataset = []

for year in os.listdir(dataset_path):
    for month in os.listdir('/'.join([dataset_path, year])):
        for f in os.listdir('/'.join([dataset_path, year, month])):
            if f.endswith('.csv'):
                learning_dataset = learning_dataset + process_csv('/'.join([dataset_path, year, month, f]))

# uzupełnienie brakujących danych ich uśrednieniem

In [3]:
import numpy as np
learning_dataset_np = np.array(learning_dataset, dtype=float)
means = np.nanmean(learning_dataset_np, axis=0)
means[np.isnan(means)] = 0
for i in range(means.size):
    learning_dataset_np[:,i][np.isnan(learning_dataset_np[:,i])] = means[i]

<ipython-input-3-22ae9804eb3c>:3: RuntimeWarning: Mean of empty slice
  means = np.nanmean(learning_dataset_np, axis=0)


# analiza przygotowanych danych za pomocą OneClassSVM

In [ ]:
from sklearn.svm import OneClassSVM
import time
print(time.time())
OSVMclf = OneClassSVM(gamma = 'auto').fit(learning_dataset_np)
print(time.time())

1623506550.025873


# wczytywanie danych z dwóch miesięcy przed awarią

In [ ]:
import os
dataset_path = './datasets_incoming_accident/LK3 045L'
test_dataset = []

for year in os.listdir(dataset_path):
    for month in os.listdir('/'.join([dataset_path, year])):
        for f in os.listdir('/'.join([dataset_path, year, month])):
            if f.endswith('.csv'):
                test_dataset = test_dataset + process_csv('/'.join([dataset_path, year, month, f]))

In [ ]:
import numpy as np
test_dataset_np = np.array(test_dataset, dtype=float)
means = np.nanmean(test_dataset_np, axis=0)
means[np.isnan(means)] = 0
for i in range(means.size):
    test_dataset_np[:,i][np.isnan(test_dataset_np[:,i])] = means[i]

In [ ]:
dd = OSVMclf.predict(test_dataset_np)

In [ ]:
cc = np.count_nonzero(dd == -1) / len(dd)

In [ ]:
dd2 = OSVMclf.predict(learning_dataset_np[:1000])

In [ ]:
np.count_nonzero(dd2 == -1) / len(dd2)

# określenie thresholdu, po przekroczeniu którego uznajemy próbkę za dotkniętą anomalią

In [ ]:
from numpy import quantile, where
scores_thresh_calc = OSVMclf.score_samples(learning_dataset_np)
thresh = quantile(scores_thresh_calc, 0.01)

# Szansa wystąpienia usterki
Niestety, w obecnym modelu przypomina trochę wróżenie z fusów. Potrzebna zmiana użytej metody przy zachowaniu tego samego podejścia.

In [ ]:
scores = OSVMclf.score_samples(test_dataset_np[index])
index = where(scores<=thresh)
len(test_dataset_np[index]) / len(test_dataset_np[index])